In [1]:
# House Cut 房源裁剪
import pandas as pd


origin_path = "./data/hd/dt_20250508.csv"
df = pd.read_csv(origin_path)
print("原始数量", len(df))
df.head(1)

原始数量 11904


,Unnamed: 0,house_id,title,district,community,position,tax,total_price,unit_price,link,hourse_type,hourse_size,direction,fitment,level,build_type
0,0,101124280562,人和育华-天鸿天鸿旁业主诚心出售,丛台区,阳光小区,苏曹商圈,-,110.0,9010.0,https://hd.lianjia.com/ershoufang/101124280562...,3室2厅,122.1,南 北,其他,6层,板楼


In [2]:
def is_high_level(row):
    # 是否层高较高
    if type(row) != dict:
        row = row.to_dict()
    if "高" in row["level"]:
        return 1
    return 0

df["is_high_level"] = df.apply(lambda row: is_high_level(row), axis=1)
df[df.is_high_level==1].level.value_counts()[:10]

level
高楼层(共6层)     1319
高楼层(共18层)     459
高楼层(共33层)     459
高楼层(共32层)     255
高楼层(共34层)     226
高楼层(共30层)     182
高楼层(共28层)     179
高楼层(共7层)      174
高楼层(共26层)     168
高楼层(共31层)     164
Name: count, dtype: int64

In [3]:
def is_unit_price_high(row):
    # 单价 > 1w2
    if type(row) != dict:
        row = row.to_dict()
    try:
        unit_price = float(row["unit_price"])
        
        if unit_price > 12000:
            return 1
    except:
        pass
    return 0

df["is_unit_price_high"] = df.apply(lambda row: is_unit_price_high(row), axis=1)
df[df.is_unit_price_high==1].unit_price.value_counts()

unit_price
12500.0    22
15000.0    10
12038.0    10
12174.0     9
16000.0     8
           ..
15808.0     1
15289.0     1
17772.0     1
21250.0     1
16529.0     1
Name: count, Length: 930, dtype: int64

In [4]:
def is_total_price_high(row):
    # 总价>130w
    if type(row) != dict:
        row = row.to_dict()
    try:
        price = float(row["total_price"])
        if price > 130:
            return 1
    except:
        pass
    return 0

df["is_total_price_high"] = df.apply(lambda row: is_total_price_high(row), axis=1)
df[df.is_total_price_high==1].total_price.value_counts()

total_price
135.0    183
145.0    131
150.0    128
140.0    116
155.0     89
        ... 
219.0      1
163.2      1
134.5      1
135.7      1
224.0      1
Name: count, Length: 213, dtype: int64

In [5]:
origin_path = origin_path.replace(".csv", "_cut.csv")
dest_df = df[
    (df.is_high_level==0) # 非高楼层
    & (df.is_unit_price_high == 0) # 单价<1w2
    & (df.is_total_price_high == 0) # 总价<130w
]
dest_df.to_csv(origin_path, index=False, encoding='utf-8-sig')
print("裁剪后数量", len(dest_df))

裁剪后数量 5431
